In [4]:
import pickle
import itertools
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
with open("curves/anotation_new_label_140.curves", "rb") as pickle_file:
    curve_fragment = pickle.load(pickle_file)

In [9]:
cfs = [[] for i in range(46)]

for cols in curve_fragment:
    for i in range(len(cols)):
        cfs[i] += cols[i]

In [10]:
_cfs = []
for i in cfs:
    if len(i) != 0:
        _cfs += i
cfs = _cfs

In [11]:
len(cfs)

190958

In [12]:
_cfs = []
for i in range(len(cfs)):
    if i%10 == 0:
        _cfs += cfs[i].tolist()
cfs = _cfs

In [13]:
pcd = o3d.io.read_point_cloud("polygon/GmJMC025_02_ds_denoise-Copy1.ply")

In [14]:
geoms = [pcd]
#geoms.append(edges_to_lineset(mesh, edges, (1, 0, 0)))

In [15]:
#o3d.visualization.draw_geometries(geoms)

In [16]:
def rotation_mat(angle):
    Rx = np.array([[1,0,0],
                 [0, np.cos(angle[0]), -np.sin(angle[0])],
                 [0, np.sin(angle[0]), np.cos(angle[0])]])

    Ry = np.array([[np.cos(angle[1]), 0, np.sin(angle[1])],
                 [0,1,0],
                 [-np.sin(angle[1]), 0, np.cos(angle[1])]])

    Rz = np.array([[np.cos(angle[2]), -np.sin(angle[2]), 0],
                 [np.sin(angle[2]), np.cos(angle[2]), 0],
                 [0,0,1]])
    return Rz@Rx@Ry

angle = [np.pi, np.pi, 0] # pcd.rotate()
#angle = [-np.pi/2, np.pi/2, np.pi/2]
R = rotation_mat(angle)
R_mirror = np.array([[-1,0,0],
                    [0,1,0],
                    [0,0,1]])

In [17]:
np.asarray(pcd.colors)

array([[0.7372549 , 0.74117647, 0.13333333],
       [0.89019608, 0.46666667, 0.76078431],
       [0.85882353, 0.85882353, 0.55294118],
       ...,
       [0.7372549 , 0.74117647, 0.13333333],
       [0.85882353, 0.85882353, 0.55294118],
       [0.85882353, 0.85882353, 0.55294118]])

In [19]:
curve_fragment = np.array((curve_fragment[0],curve_fragment[2],curve_fragment[1]))

C:\Users\murata\anaconda3\envs\py37cv\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [23]:
flatten_curve_frag = list(itertools.chain.from_iterable(cfs))
#flatten_curve_frag = (np.linalg.inv(R_mirror)@np.linalg.inv(R)@flatten_curve_frag.T).T
flatten_curve_frag = np.array(cfs)
point_num = flatten_curve_frag.shape[0]
curve_pcd = o3d.geometry.PointCloud()
curve_pcd.points = o3d.utility.Vector3dVector(flatten_curve_frag)
curve_pcd.colors = o3d.utility.Vector3dVector(np.repeat(np.array((0, 1, 0)).reshape(1,-1), point_num, axis=0))
geoms.append(curve_pcd)

In [24]:
o3d.visualization.draw_geometries(geoms)